In [1]:
from bs4 import BeautifulSoup
import requests

In [14]:
def download_data(url, proxy=None):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:79.0) Gecko/20100101 Firefox/79.0",
        "Accept": "*/*",
        "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br"}

    s = requests.Session()
    page = s.get(url,
                 headers=headers,
                 proxies={"http": proxy, "https": proxy}, stream=True
                )
    # Stream=True => In chunked transfer encoding, the data stream is divided into a series of non-overlapping "chunks". The chunks are sent out independently of one another by the server. Hope this answers the question.
#     if page.status_code == 429:
#         print("Blocked due to many requests")
#     if page.status_code == 403:
#         raise Exception("Url is forbidden")

    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [15]:
page = download_data("https://www.youtube.com/watch?v=8kEOoLomYzA")

In [446]:
print(page.prettify())

NameError: name 'page' is not defined

# Selenium

In [499]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [500]:
options = Options()
options.add_argument("--headless")

In [513]:
service = Service(executable_path="./geckodriver")
driver = webdriver.Firefox(service=service, options=None)

In [502]:
driver.get("https://www.youtube.com/watch?v=8kEOoLomYzA")

# get all sentences 

In [283]:
#url = "ytd-menu-service-item-renderer.style-scope:nth-child(2) > tp-yt-paper-item:nth-child(1) > yt-formatted-string:nth-child(2)"
url = "ytd-menu-renderer.ytd-video-primary-info-renderer > yt-icon-button:nth-child(3) > button:nth-child(1)"
button = driver.find_element_by_css_selector(url)

/tmp/ipykernel_1120332/3729645192.py:3: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_element_by_css_selector(url)


In [284]:
button.click()

hidden buttion

In [285]:
url = "ytd-menu-service-item-renderer.style-scope:nth-child(2) > tp-yt-paper-item:nth-child(1) > yt-formatted-string:nth-child(2)"
button = driver.find_element_by_css_selector(url)

/tmp/ipykernel_1120332/876996630.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_element_by_css_selector(url)


In [286]:
button.click()

In [287]:
css_selector = "html body ytd-app div#content.style-scope.ytd-app ytd-page-manager#page-manager.style-scope.ytd-app ytd-watch-flexy.style-scope.ytd-page-manager.hide-skeleton div#columns.style-scope.ytd-watch-flexy div#secondary.style-scope.ytd-watch-flexy div#secondary-inner.style-scope.ytd-watch-flexy div#panels.style-scope.ytd-watch-flexy ytd-engagement-panel-section-list-renderer.style-scope.ytd-watch-flexy"
transcript = driver.find_element_by_css_selector(css_selector)

/tmp/ipykernel_1120332/1986423723.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  transcript = driver.find_element_by_css_selector(css_selector)


In [288]:
#xpath = "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/div[1]/ytd-engagement-panel-section-list-renderer[4]/div[2]/ytd-transcript-renderer/div[2]/ytd-transcript-search-panel-renderer/div[2]/ytd-transcript-segment-list-renderer/div[1]/ytd-transcript-segment-renderer"
# short version
xpath = "//div[contains(@class, 'segment style-scope ytd-transcript-segment-renderer')]"
teams = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, xpath)))

In [289]:
all_text = []
for x in teams:
    all_text.append(x.text.split('\n')[1])
    #break

In [290]:
#all_text = " ".join(all_text)

In [291]:
import re
for text in all_text:
    res = [(x.group(0), x.span()) for x in list(re.finditer("\s(\w+)?ху(\w*)?\s", text))]
    if res:
        print(res)

[(' хуйню ', (20, 27))]
[(' хуйню ', (6, 13))]
[(' нахуй ', (14, 21))]
[(' хуй ', (19, 24))]
[(' нюху ', (13, 19))]
[(' нахуй ', (12, 19))]
[(' хуячить ', (19, 28))]
[(' хуйню ', (16, 23))]
[(' нахуя ', (21, 28))]
[(' нахуй ', (9, 16))]
[(' нахуй ', (6, 13))]
[(' ахуенно ', (26, 35))]
[(' худший ', (7, 15))]
[(' хуже ', (24, 30))]
[(' хуйнёй ', (26, 34))]


# single example

In [102]:
xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[2]/div/div[1]/ytd-engagement-panel-section-list-renderer[4]/div[2]/ytd-transcript-renderer/div[2]/ytd-transcript-search-panel-renderer/div[2]/ytd-transcript-segment-list-renderer/div[1]/ytd-transcript-segment-renderer[1]/div'
descr = transcript.find_element_by_xpath(xpath)

/tmp/ipykernel_1120332/1599345750.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  descr = transcript.find_element_by_xpath(xpath)


In [106]:
descr.text

'0:00\nпринципе можно конечно'

# get all videos from the channel 

In [492]:
driver.get("https://www.youtube.com/c/MadHighlights/videos")

In [510]:
# test
xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer'
#xpath = '//div[contains(@class,"stroke style-scope yt-interaction")]'
video_preview = driver.find_elements_by_xpath(xpath)

/tmp/ipykernel_1120332/1870542600.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  video_preview = driver.find_elements_by_xpath(xpath)


In [511]:
len(video_preview)

1008

In [512]:
for x in video_preview:
    print(x.text)

45:23
Мэддисон катает шары в Elden Ring
28K views
5 hours ago
29:15
Мэддисона забуллил взбесившийся волк в Elden Ring
139K views
6 days ago
35:19
Мэддисон и очередной дед в Elden Ring
142K views
7 days ago
30:22
Мэддисон и дед номер два в Elden ring
140K views
9 days ago
34:43
Мэддисон и его первый дед в Elden Ring
185K views
10 days ago
39:24
Мэддисон и Welovegames отмечают десятилетие Warface
139K views
2 weeks ago
38:16
Мэддисон играет в Макса Пейна НА РУЛЕ
282K views
4 weeks ago
1:15:09
Мэддисон играет в Lost Ark и прощается с Твичем
193K views
1 month ago
42:01
Мэддисон нашел персонажа которого озвучил в Lost Ark
154K views
1 month ago
1:06:09
Мэддисон смотрит Музыкальный Ринг (Круг vs Трофим ), Шоу Окна, и пробует уральский майонез
248K views
1 month ago
43:35
Мэддисон играет в божественный Lost Ark и открывает собственный бар
142K views
1 month ago
51:17
Мэддисон и актуальные темы новой реальности /madcast
324K views
1 month ago
37:18
Мэддисон играет в Ведьмака с плохими выборам

40:22
Мэддисон собирает профсоюзную сходку в GTA5 RP
80K views
7 months ago
36:15
Мэддисон смотрит боевики 90х
291K views
7 months ago
45:50
Мэддисон с подписчиками выбирают игру для прохождения
165K views
7 months ago
19:22
Мэддисон нашел консоль с ретро играми
145K views
7 months ago
44:07
Мэддисон тюнингует Ладу Весту в GTA5 RP
86K views
8 months ago
26:31
Мэддисон играет в Pathfinder: Wrath of the Righteous
123K views
8 months ago
53:34
Мэддисон собирает jpop группу в Idol Manager + то что было после стрима
153K views
8 months ago
1:15:42
Мэддисон пробует блинчики в GRA RP
132K views
8 months ago
34:53
Мэддисон смотрит игры присланные донатерами
219K views
8 months ago
34:20
Мэддисон добрался до бара в Genshin Impact, и сделал обзор на всех персонажей
163K views
8 months ago
33:26
Мэддисон играет в GTA 5 RP
104K views
8 months ago
44:15
Мэддисон надел новое платье на сабдей (Stronghold, Baldur’s Gate, The Suicide of Rachel Foster)
247K views
8 months ago
35:02
Мэддисон смотрит Game

30:47
Мэддисон и Хованский похищают прокурора в GTA 5 RP
103K views
1 year ago
30:34
Мэддисон строит стену в GTA 5 RP
93K views
1 year ago
3:55
Ухожу в шансон
149K views
1 year ago
28:41
Мэддисон и Хованский снимают "Схватку" в GTA 5 RP
112K views
1 year ago
26:41
Мэддисон отчитывается перед начальством в GTA 5 RP
108K views
1 year ago
32:44
Мэддисон ловит на наживку в GTA 5 RP
119K views
1 year ago
29:23
Мэддисон создает видимость в GTA 5 RP
139K views
1 year ago
1:06:39
Мэддисон сидит в таверне с чатом /madcast
151K views
1 year ago
9:57
Консоли снова опозорились
158K views
1 year ago
38:03
Мэддисон прошел The Elder Scrolls III: Morrowind
312K views
1 year ago
15:31
Мэддисон дропнул Disco Elysium
212K views
1 year ago
19:23
Мэддисону строят комнату грязи в Rust
152K views
1 year ago
1:03:21
Форд Мустанг, Football Manager 21, La2, Call of LDPR
160K views
1 year ago
49:05
Куда идти с твитча после пермача
234K views
1 year ago
9:19
Самый честный обзор на Call Of Ldpr
285K views
1 year a

24:57
Мэддисон вернулся из деревни /madcast
183K views
1 year ago
21:26
Мэддисон прошел Watch Dogs
165K views
1 year ago
21:45
Мэддисон слушает новую богемскую рапсодию в Watch Dogs
189K views
1 year ago
22:39
Мэддисон встретил сильную женщину в Skyrim
252K views
1 year ago
27:27
Мэддисон во дворца Skyrim
261K views
1 year ago
24:24
Мэддисон спасает Тодда Говарда в Skyrim
255K views
1 year ago
37:37
Мэддисон обозревает финал The Last of Us 2
471K views
1 year ago
30:31
Мэддисон не знает жалости в The Last of Us 2
232K views
1 year ago
25:33
Мэддисон мстит в The Last of Us 2
201K views
1 year ago
37:01
Мэддисон не грустит в The Last of Us 2
304K views
1 year ago
24:14
Мэддисон занимается математическим алкоголизмом в Watch Dogs
194K views
1 year ago
21:17
Мэддисон становится ассасином в Skyrim
277K views
1 year ago
24:52
Мэддисон помогает школьникам в Skyrim
251K views
1 year ago
21:58
Мэддисон тренирует новую напарницу Skyrim
246K views
1 year ago
19:30
Мэддисону не рады в Skyrim
306K 

29:34
Мэддисон, предвыборная программа, ответ Карине /madcast
227K views
2 years ago
28:11
Мэддисон про сериал "Ведьмак" /madcast
238K views
2 years ago
21:46
Мэддисон смотрит "Брат 3" и поет Мишу Круга
167K views
2 years ago
14:29
Мэддисон играет в Trepang2 - Не хватает таких игр
210K views
2 years ago
1:31:27
Мэддисон, лучшие моменты 2019
1.6M views
2 years ago
29:54
Мэддисон не хочет умирать - Усачев, Мизулина, Виндоус, Туалет
220K views
2 years ago
18:44
Мэддисон не берет деда в таргет в Bloodborne #2
152K views
2 years ago
29:51
Мэддисон и Кармик Коала комментируют The Game Awards 2019
192K views
2 years ago
22:05
Мэддисон про вступление в ЛДПР, о своем диагнозе, ответ Камикадзе, дисквалификация России
243K views
2 years ago
26:46
Мэддисон первый раз играет в Bloodborne
272K views
2 years ago
13:49
Мэддисон играет в Red Orchestra 2
139K views
2 years ago
34:53
Мэддисона заставляют ходить в Vampire: The Masquerade – Bloodlines
285K views
2 years ago
18:35
Мэддисон ползает в Dishono

36:59
Мэддисон покупает BMW X5 и отвечает на донаты
260K views
3 years ago
25:56
Мэддисон смотрит тренды ютуба - О СОВКЕ, ХАТА МАРЬЯНЫ РО, ЖЕНСКИЕ СКЕТЧИ
269K views
3 years ago
36:05
Мэддисон в GTA RP - Большая Потеря, Шериф Хортон #4
131K views
3 years ago
30:27
Мэддисон в GTA RP - Соревновательный день, Шериф Хортон #3
121K views
3 years ago
25:43
Мэддисон в GTA RP - Падение Украинского Дирижабля, Шериф Хортон #2
155K views
3 years ago
19:28
Мэддисон смотрит тренды ютуба - Слава КПСС, ЕРЖАН, СОНИК В КИНО
402K views
3 years ago
35:38
Мэддисон в GTA RP - Король Севера, Шериф Хортон #1
167K views
3 years ago
15:15
Мэддисон строит коммунизм в игре Soviet Republic
345K views
3 years ago
22:01
Мэддисон и Смайл - Кулинарный стрим, и пивной челендж
124K views
3 years ago
29:30
Мэддисон играет в GTA RP - Улицы сгоревших автомобилей episode 4
114K views
3 years ago
44:50
Мэддисон играет в GTA RP - Улицы сгоревших автомобилей, episode 3
159K views
3 years ago
41:20
Мэддисон играет в GTA RP - Ул

21:59
200K ДОНАТ, ЛАСКА СТРИМЕРША, ЗЛОЙ СИГАЛ - Стримхата, day 4/1
71K views
3 years ago
35:49
Стримхата, day 3 - Гитмурик клянчит лайки, Лучшие болельщики в истории, Мэд подсасывает факеру
141K views
3 years ago
25:20
Стримхата, day 2 - Командный День
139K views
3 years ago
10:16
Стримхата, day zero - Футбольчик
104K views
3 years ago
37:03
Стримхата, day 1 - Званый Ужин
214K views
3 years ago
23:28
Мэддисон и турик по FIFA
217K views
3 years ago
27:23
Мэддисон³ - Fallout 3 - Выпуск 3
397K views
3 years ago
31:12
Мэддисон играет в Чушки 3 - Ищем fallout
644K views
3 years ago
29:31
Мэддисон играет в Fallout 3 #1 - Здравствуй дочь
442K views
3 years ago
45:46
Мэддисон играет в Fallout 2 (2018)
523K views
3 years ago
29:13
Мэддисон тащит сборную России к победе в FIFA WORLD CUP RUSSIA 2018
718K views
3 years ago
36:33
Мэддисон комментирует E3 - Sony
202K views
3 years ago
20:14
Мэддисон комментирует E3 - PC gaming show
104K views
3 years ago
31:23
Мэддисон комментирует E3 - Ubisoft, Squ

33:52
Мэддисон играет в Ведьмак 3, day 3
266K views
4 years ago
25:26
Мэддисон играет в Ведьмак 3, day 2
261K views
4 years ago
28:06
Мэддисон играет в "Ведьмак 3"
332K views
4 years ago
33:11
Мэддисон играет в L.A. Noire Remastered, day 4
135K views
4 years ago
35:11
Мэддисон играет в L.A. Noire Remastered, day 3
156K views
4 years ago
27:49
Maddyson Subday: Cuphead, The movies, новелла с Санбоем
221K views
4 years ago
20:53
Мэддисон ностальгирует в Gran Turismo Sport
149K views
4 years ago
10:35
Мэддисон играет в Human Fall Flat
142K views
4 years ago
25:14
Мэддисон играет в Warface
307K views
4 years ago
24:37
Мэддисон играет в LA Noire Remastered, day 2
142K views
4 years ago
27:00
Мэддисон играет в LA Noire Remastered
272K views
4 years ago
22:06
Maddyson Subday: theHunter, Dark Souls, Doki Doki
145K views
4 years ago
25:44
Мэддисон играет в Call of Duty: WWII
431K views
4 years ago
11:22
Мэддисон играет в Мафию онлайн, Квиплеш
60K views
4 years ago
37:18
Maddyson GTA RP / Банда ц

29:17
Мэддисон играет в GTA 5 от первого лица #2
328K views
5 years ago
22:04
Мэддисон играет в Сталкер Мизери #2
395K views
5 years ago
22:07
Мэддисон играет в Final Fantasy 15
247K views
5 years ago
23:32
Мэддисон играет в Watch Dogs 2
457K views
5 years ago
23:39
Мэддисон играет в Rust с Кейком и Дрэдом
131K views
5 years ago
30:15
Мэддисон играет в Хитман 2016
404K views
5 years ago
20:27
Мэддисон играет в Quake 2 с Гитманом
75K views
5 years ago
21:22
Мэддисон играет в Сталкер мизери
615K views
5 years ago
27:40
Мэддисон играет в GTA 5 от первого лица #1
624K views
5 years ago
21:43
Мэддисон играет в GTA 5 Трон/Аста ла виста
270K views
5 years ago
25:40
Мэддисон играет в My Summer Car
652K views
5 years ago
21:33
Мэддисон смотрит видосики #3
958K views
5 years ago
36:17
Мэддисон смотрит видосики #2
616K views
5 years ago
12:12
Мэд играет в H1Z1: King of the Kill
55K views
5 years ago
30:15
Мэддисон смотрит видосики
1.8M views
5 years ago
23:19
Мэддисон играет в Dishonored 2
362K v

1:26
Call of Duty: Black Ops III - Страшная Смерть
25K views
6 years ago
8:30
Нарезка от 4.10.2015 - Counter-Strike: Global Offensive
28K views
6 years ago
3:33
мэддисон - Джерико вернулся в Мегатонну
64K views
6 years ago
5:24
Fallout 3 - Похороны Харона
59K views
6 years ago
12:58
Мэддисон играет в Darkness
68K views
6 years ago
11:25
Таксуем в GTA 5
113K views
6 years ago
15:41
Мэддисон играет в Arma 3 life [Бордель]
75K views
6 years ago
13:27
Мэддисон играет в Arma 3 life - Подстава
72K views
6 years ago
30:08
Мэддисон играет в Arma 3 life [Тувинец]
140K views
6 years ago
8:27
Мэддисон играет в Arma 3 life - Угнать за 60 секунд
42K views
6 years ago
20:36
Мэддисон играет в Assassin's Creed: Syndicate day2
167K views
6 years ago
14:29
Мэддисон играет в Assassin's Creed: Syndicate
186K views
6 years ago
40:50
Мэддисон играет в Arma 3 Life [Полицейский беспредел]
95K views
6 years ago
12:54
Мэддисон играет в Hitman: Absolution
205K views
6 years ago
14:56
Мэддисон, дикие рофлы в GTA 

# Scroll to get all videos

In [503]:
from selenium.webdriver.common.action_chains import ActionChains

In [504]:
    from selenium.webdriver.common.keys import Keys

In [505]:
driver.get("https://www.youtube.com/c/MadHighlights/videos")

In [506]:
import time

In [508]:
xpath = '//*[@id="contents"]'
element = driver.find_element_by_xpath(xpath) # the element you want to scroll to 

/tmp/ipykernel_1120332/3837978472.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element = driver.find_element_by_xpath(xpath) # the element you want to scroll to


In [509]:
prev_page_pos = 0
while True:
    #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    driver.find_element_by_tag_name('body').send_keys(Keys.END) # triggers AJAX request to load more users. observed that loading 10 users at a time.
    new_page_pos = str(driver.execute_script('return window.pageYOffset;'))
    print(prev_page_pos, new_page_pos)
    if prev_page_pos == new_page_pos:
        break
    prev_page_pos = new_page_pos
    time.sleep(1)

/tmp/ipykernel_1120332/1628361123.py:4: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  driver.find_element_by_tag_name('body').send_keys(Keys.END) # triggers AJAX request to load more users. observed that loading 10 users at a time.


0 0
0 707
707 1787
1787 2867
2867 3947
3947 5027
5027 6107
6107 7187
7187 8267
8267 9347
9347 10427
10427 11507
11507 12587
12587 13667
13667 14747
14747 15827
15827 16907
16907 17987
17987 19067
19067 20189
20189 21227
21227 22307
22307 23387
23387 24467
24467 25547
25547 26627
26627 27707
27707 28787
28787 29867
29867 30947
30947 32052
32052 33107
33107 34187
34187 35451
35451 35855
35855 35855


# close

In [315]:
driver.close() #?
driver.quit()

InvalidSessionIdException: Message: Tried to run command without establishing a connection
